In [18]:
import shutil
import os
import xml.etree.ElementTree as ET

In [19]:
dataset_path = "data/LogoDet-3K"
output_img_dir = "data/preprocessed/images"
output_label_dir = "data/preprocessed/labels"

In [20]:
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

In [21]:
class_dict = {}
class_id_counter = 0

Переводим файлы XML в txt и формируем новую структуру хранения данных:

-preprocessed \
    -images \
        -category1
            -brand1_ph_1.jpg
            -brand1_ph_n.jpg
            -brand2_ph_1.jpg
            -brand2_ph_n.jpg
        -category2
            -brand3_ph_1.jpg
            -brand3_ph_n.jpg
            -brand4_ph_1.jpg
            -brand5_ph_n.jpg
    -labels
        -category1
            -brand1_ph_1.xml
            -brand1_ph_n.xml
            -brand2_ph_1.xml
            -brand2_ph_n.xml
        -category2
            -brand3_ph_1.xml
            -brand3_ph_n.xml
            -brand4_ph_1.xml
            -brand5_ph_n.xml

In [23]:
# Перебираем категории и бренды
for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)
    
    for brand in os.listdir(category_path):
        # Если бренд ещё не добавлен в словарь, добавляем его с уникальным class_id
        if brand not in class_dict:
            class_dict[brand] = class_id_counter
            class_id_counter += 1

        brand_path = os.path.join(category_path, brand)
        
        # Перебираем все изображения и соответствующие XML файлы
        for filename in os.listdir(brand_path):
            if filename.endswith(".xml"):
                xml_path = os.path.join(brand_path, filename)
                image_path = os.path.join(brand_path, filename.replace(".xml", ".jpg"))
                
                try:
                    # Загружаем XML
                    tree = ET.parse(xml_path)
                    root = tree.getroot()

                    # Получаем размеры изображения
                    img_width = int(root.find("size/width").text)
                    img_height = int(root.find("size/height").text)
                    
                    # Получаем все объекты в изображении
                    labels = []
                    for obj in root.findall("object"):
                        class_name = obj.find("name").text.strip()  # Убираем лишние пробелы

                        # Проверка на наличие бренда в словаре
                        if class_name not in class_dict:
                            print(f"Бренд {class_name} не найден в словаре, добавляем его.")
                            class_dict[class_name] = class_id_counter
                            class_id_counter += 1
                        
                        xmin = int(obj.find("bndbox/xmin").text)
                        ymin = int(obj.find("bndbox/ymin").text)
                        xmax = int(obj.find("bndbox/xmax").text)
                        ymax = int(obj.find("bndbox/ymax").text)
                        
                        # Нормализуем координаты
                        x_center = (xmin + xmax) / 2 / img_width
                        y_center = (ymin + ymax) / 2 / img_height
                        width = (xmax - xmin) / img_width
                        height = (ymax - ymin) / img_height
                        
                        # Используем class_id из словаря
                        class_id = class_dict[class_name]

                        # Сохраняем в список
                        labels.append(f"{class_id} {x_center} {y_center} {width} {height}")
                    
                    # Генерация уникальных имен файлов
                    base_filename = f"{brand}_ph_{filename.split('.')[0]}"
                    unique_image_filename = f"{base_filename}.jpg"
                    unique_xml_filename = f"{base_filename}.xml"
                    unique_label_filename = f"{base_filename}.txt"

                    # Создаем подкаталоги для категории и бренда в папках images и labels
                    category_img_dir = os.path.join(output_img_dir, category)
                    category_label_dir = os.path.join(output_label_dir, category)

                    os.makedirs(category_img_dir, exist_ok=True)
                    os.makedirs(category_label_dir, exist_ok=True)

                    # Переносим изображение в папку images
                    dest_image_path = os.path.join(category_img_dir, unique_image_filename)
                    shutil.copy(image_path, dest_image_path)
                    
                    # Записываем метки в папку labels
                    unique_label_filepath = os.path.join(category_label_dir, unique_label_filename)
                    with open(unique_label_filepath, "w") as label_file:
                        label_file.write("\n".join(labels))

Бренд Alcatel-2 не найден в словаре, добавляем его.
Бренд alpinestars-2 не найден в словаре, добавляем его.
Бренд atari 2600-2 не найден в словаре, добавляем его.
Бренд Baume et Mercier-2 не найден в словаре, добавляем его.
Бренд Chipotle Mexican Grill-2 не найден в словаре, добавляем его.
Бренд Ck Calvin Klein-2 не найден в словаре, добавляем его.
Бренд ERKE-2 не найден в словаре, добавляем его.
Бренд Glock-2 не найден в словаре, добавляем его.
Бренд Harry Winston-2 не найден в словаре, добавляем его.
Бренд Harvest Moon не найден в словаре, добавляем его.
Бренд Heat-2 не найден в словаре, добавляем его.
Бренд Hummel-2 не найден в словаре, добавляем его.
Бренд Hurley-2 не найден в словаре, добавляем его.
Бренд Isabel Maran-2 не найден в словаре, добавляем его.
Бренд INOHERB-2 не найден в словаре, добавляем его.
Бренд jlindeberg-2 не найден в словаре, добавляем его.
Бренд Joma-2 не найден в словаре, добавляем его.
Бренд Karl Kani-2 не найден в словаре, добавляем его.
Бренд KATE SPADE-2 

In [27]:
class_dict['Alcatel-1']

615